# Machine Learning Models
## Load Libraries

In [1]:
import pandas as pd
import pytimetk as tk

import pycaret.classification as clf
import pycaret.regression as reg

In [2]:
# constants 
profit_margin = 0.15 #15% Profit on Products

In [3]:
# Helper Function
from datetime import datetime

# differences in months.
def diff_month(d1, d2):
    if ((d1.year - d2.year) * 12 + d1.month - d2.month) == 0:
        return 1
    else:
        return (d1.year - d2.year) * 12 + d1.month - d2.month

In [4]:
# Helper Functions

# Converting Churn Rates:
###########################################################
# To convert annual churn to a monthly churn rate:
def churn_mthly_2_annual(mthly_churn_rate):
    return (1.0-((1.0-mthly_churn_rate)**(12)))

# To convert monthly churn to an annual churn rate:
def churn_annual_2_mthly(annual_churn_rate):
    return (1.0-((1.0-annual_churn_rate)**(1/12.0)))

# To convert churn rate to any period:
def churn_any_period_2_mthly(period_churn_rate, num_mths_in_period):
    return (1.0-((1.0-period_churn_rate)**(1/num_mths_in_period)))


# Customer Lifetime Value
###########################################################
# Customer Lifetime Months 
def cust_lifetime_in_mths(MonthlyChurnRate):
    return (1 / MonthlyChurnRate)

# Customer Lifetime Years
def cust_lifetime_in_years(MonthlyChurnRate):
    return (cust_lifetime_in_mths(MonthlyChurnRate) / 12.0)

# Monthly Churn Rate % from customer lifetime in months.
def mthly_churn_rate_from_mthly_lifetime(CustomerLifetimeMonths):
    return (1 / CustomerLifetimeMonths)

# Monthly Churn Rate % from customer lifetime in years.
def mthly_churn_rate_from_yearly_lifetime(CustomerLifetimeYears):
    return (1 / (CustomerLifetimeYears * 12))
    
# Decay
###########################################################
# Want to know how many customers will remain after 6, 12, 24 months?
# Here is how to calculate how many customers in a cohort will be left at Month X, 
# using exponential decay.

def custs_remaining(MonthlyChurnRate, StartingCohortCount, NumberOfMonthsInTheFuture):
    return (StartingCohortCount * ((1 - MonthlyChurnRate) ^ NumberOfMonthsInTheFuture))


# Calculate Churn
###########################################################
# Annual Churn Rate
def annual_churn_rate(CohortYearStartCount, CohortYearEndCount):
    return ((CohortYearStartCount - CohortYearEndCount) / CohortYearStartCount)

# Monthly Churn Rate
def mthly_churn_rate(CohortMonthStartCount, CohortMonthEndCount):
    return ((CohortMonthStartCount - CohortMonthEndCount) / CohortMonthStartCount)

# Any Period Churn Rate
def any_period_churn_rate(CohortPeriodStartCount, CohortPeriodEndCount):
    return ((CohortPeriodStartCount - CohortPeriodEndCount) / CohortPeriodStartCount)


## Import Data

In [5]:
# Accomodate raw path to variables
raw_customer, raw_orders = "./input/customers.csv", "./input/orders.csv"
raw_products, raw_sales = "./input/products.csv", "./input/sales.csv"

# Read-in data
customer, order = pd.read_csv(raw_customer), pd.read_csv(raw_orders)
product, sales_data = pd.read_csv(raw_products), pd.read_csv(raw_sales)

In [6]:
print(order['order_date'].min())
print(order['order_date'].max())

2021-1-1
2021-9-9


In [7]:
order.head(3)

,order_id,customer_id,payment,order_date,delivery_date
0,1,64,30811,2021-8-30,2021-09-24
1,2,473,50490,2021-2-3,2021-02-13
2,3,774,46763,2021-10-8,2021-11-03


In [8]:
df = order.copy()
df['order_date'] = pd.to_datetime(df['order_date'])

In [9]:
# Select a cohort
df1 = df[(df['order_date']>=datetime(2021, 4, 1))&
         (df['order_date']<datetime(2021, 5, 1))]
df1.head(3)

,order_id,customer_id,payment,order_date,delivery_date
6,7,626,37666,2021-04-05,2021-04-11
7,8,58,28484,2021-04-12,2021-05-01
20,21,124,11261,2021-04-04,2021-04-17


## EDA

In [10]:
(df[['order_date','payment']].summarize_by_time(
    date_column = 'order_date',
    value_column = 'payment',
    agg_func = 'sum',
    freq = 'M'
)
 .plot_timeseries('order_date','payment'))

In [11]:
(df[['order_date','payment']].summarize_by_time(
    date_column = 'order_date',
    value_column = 'payment',
    agg_func = 'sum',
    freq = 'D'
)
 .plot_timeseries('order_date','payment'))

In [12]:
(df1[['order_date','payment']].summarize_by_time(
    date_column = 'order_date',
    value_column = 'payment',
    agg_func = 'sum',
    freq = 'D'
)
 .plot_timeseries('order_date','payment'))

# Machine Learning Models
### Task:
- What will be the customers spend in the next 90 days? (Regression Problem)
- What is the probablilty for a customer to make another purchase in the next 90 days? (Classification Problem) 

## ETL

In [48]:
# Set Parameters
n_days = 90
max_date = df['order_date'].max()
cutoff = max_date - pd.to_timedelta(n_days, unit='d')

In [16]:
# Train-Test Split
temporal_in_df = df[df['order_date'] < cutoff]

temporal_out_df = (df[df['order_date'] >= cutoff]
                   .query('customer_id in @temporal_in_df.customer_id'))

In [17]:
temporal_out_df.glimpse()

<class 'pandas.core.frame.DataFrame'>: 157 rows of 5 columns
order_id:       int64             [3, 6, 10, 24, 28, 36, 42, 50, 52, 84, ...
customer_id:    int64             [774, 800, 659, 469, 206, 51, 690, 454 ...
payment:        int64             [46763, 16197, 21922, 14566, 29968, 32 ...
order_date:     datetime64[ns]    [Timestamp('2021-10-08 00:00:00'), Tim ...
delivery_date:  object            ['2021-11-03', '2021-10-05', '2021-10- ...


## Feature Enginerring

### Main Features

In [22]:
# Make Targets from out data. 
targets_df = (temporal_out_df[['customer_id', 
                               'payment']]
              .groupby('customer_id')
              .sum()
              .rename({'payment':'sales_90_value'}, axis = 1)
              .assign(sales_90_flag = 1)
)

targets_df.head(3)

,sales_90_value,sales_90_flag
customer_id,,
15,50335,1
20,37821,1
28,39533,1


In [25]:
# Make Recency (Date) Features from in data
recency_features_df = (temporal_in_df[['customer_id', 
                                       'order_date']]
                       .groupby('customer_id')
                       .apply(
                           lambda x: int((x['order_date'].max() - max_date) /
                                         pd.to_timedelta(1,"day"))
                       )
                       .to_frame()
                       .set_axis({'recency'}, axis=1)
                       )

recency_features_df.head(3)

,recency
customer_id,
1,-203
7,-156
10,-229


In [26]:
# Make Frequency (Count) Features from in data.
frequency_features_df = (temporal_in_df[['customer_id', 
                                       'order_date']]
                       .groupby('customer_id')
                       .count()
                       .set_axis({'frequency'}, axis=1)
                       )

frequency_features_df.head(3)

,frequency
customer_id,
1,3
7,1
10,1


In [32]:
# Make Monetary Featues from in data.
monetary_features_df = (temporal_in_df
                       .groupby('customer_id')
                       .aggregate(
                           {
                               'payment': ['sum', 'mean']   
                           }
                       )
                       .set_axis({'sales_value_sum','sales_value_mean'}, axis=1)
                       )

monetary_features_df.head(3)

,sales_value_mean,sales_value_sum
customer_id,,
1,70389,23463.0
7,48935,48935.0
10,45626,45626.0


### Additional Features

In [50]:
# Make number of transactions in last month (28 Days) from in data.
lookback_days = 28
cutoff_lookback_days = cutoff - pd.to_timedelta(lookback_days, unit = "d")

transactions_last_month_df = (temporal_in_df[['customer_id', 
                                              'order_date']]
                              .drop_duplicates()
                              .query('order_date >= @cutoff_lookback_days')
                              .groupby('customer_id')
                              .size()
                              .to_frame()
                              .set_axis({'transactions_last_month'}, axis=1)
                              )

transactions_last_month_df.head(3)

,transactions_last_month
customer_id,
28,1
44,1
50,1


In [51]:
# Make number of transactions in last two weeks (14 Days) from in data.
lookback_days = 14
cutoff_lookback_days = cutoff - pd.to_timedelta(lookback_days, unit = "d")

transactions_last_2_weeks_df = (temporal_in_df[['customer_id', 
                                              'order_date']]
                              .drop_duplicates()
                              .query('order_date >= @cutoff_lookback_days')
                              .groupby('customer_id')
                              .size()
                              .to_frame()
                              .set_axis({'transactions_last_2_weeks'}, axis=1)
                              )

transactions_last_2_weeks_df.head(3)

,transactions_last_2_weeks
customer_id,
44,1
50,1
60,1


In [14]:
df.columns

Index(['order_id', 'customer_id', 'payment', 'order_date', 'delivery_date'], dtype='object')

In [52]:
# Make Spend in the last 2 weeks
lookback_days = 14
cutoff_lookback_days = cutoff - pd.to_timedelta(lookback_days, unit = "d")

spend_last_2_weeks_df = (temporal_in_df[['customer_id',
                                         'order_date',
                                         'payment']]
                              .drop_duplicates()
                              .query('order_date >= @cutoff_lookback_days')
                              .groupby('customer_id')
                              .sum()
                              .set_axis({'spend_last_2_weeks'}, axis=1)
                              )

spend_last_2_weeks_df.head(3)

,spend_last_2_weeks
customer_id,
44,59055
50,51943
60,28334


### Combine Features

In [53]:
# Combine Features
features_df = (pd.concat(
    [recency_features_df,
     frequency_features_df,
     monetary_features_df,
     transactions_last_month_df,
     transactions_last_2_weeks_df,
     spend_last_2_weeks_df], axis=1)
               .merge(targets_df,
                      left_index=True,
                      right_index=True,
                      how="left")
               .fillna(0)
)

features_df.head(3)

,recency,frequency,sales_value_mean,sales_value_sum,transactions_last_month,transactions_last_2_weeks,spend_last_2_weeks,sales_90_value,sales_90_flag
customer_id,,,,,,,,,
1,-203,3,70389,23463.0,0.0,0.0,0.0,0.0,0.0
7,-156,1,48935,48935.0,0.0,0.0,0.0,0.0,0.0
10,-229,1,45626,45626.0,0.0,0.0,0.0,0.0,0.0


# Regression (Spend in next 90 days)

In [54]:
reg_setup = reg.setup(
    data = features_df.drop('sales_90_flag', axis = 1),
    target='sales_90_value',
    train_size=0.8,
    normalize=True,
    session_id=123,
    verbose=True,
    log_experiment=False
)

xgb_reg_model = reg.create_model('xgboost')
reg_predictions_df = (reg
                      .predict_model(xgb_reg_model, 
                                     data=features_df)
                      .sort_values('prediction_label', ascending=False))


,Description,Value
0,Session id,123
1,Target,sales_90_value
2,Target type,Regression
3,Original data shape,"(488, 8)"
4,Transformed data shape,"(488, 8)"
5,Transformed train set shape,"(390, 8)"
6,Transformed test set shape,"(98, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,18521.5781,751156416.0000,27407.2324,-1.5574,7.5962,0.7259
1,17470.4160,544537024.0000,23335.3164,-0.3011,7.6573,0.6606
2,17283.5332,630605504.0000,25111.8594,-0.6215,7.7273,0.7782
3,21815.0176,867371840.0000,29451.1777,-0.5447,7.7800,1.0204
4,20720.9551,837738944.0000,28943.7207,-1.9068,8.2814,1.1066
5,12691.0732,333366272.0000,18258.3203,-0.2709,7.5087,0.7177
6,22136.9473,1113515392.0000,33369.3789,-0.3235,7.3705,0.7632
7,21870.5430,1050755520.0000,32415.3594,-0.3660,6.9227,0.7977
8,19736.7051,847784640.0000,29116.7422,-0.6439,7.5129,0.8559


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,4512.4653,117375944.0000,10834.0176,0.7490,5.9619,0.1970


# Classification (Spend Probability in next 90 days)

In [55]:
clf_setup = clf.setup(
    data = features_df.drop('sales_90_value', axis = 1),
    target='sales_90_flag',
    train_size=0.8,
    session_id=123,
    verbose=True,
    log_experiment=False
)

xgb_clf_model = clf.create_model('xgboost')
clf_predictions_df = (clf
                      .predict_model(xgb_clf_model, 
                                     data=features_df,
                                     raw_score=True)
                      .sort_values('prediction_score_1', ascending=False))

,Description,Value
0,Session id,123
1,Target,sales_90_flag
2,Target type,Binary
3,Original data shape,"(488, 8)"
4,Transformed data shape,"(488, 8)"
5,Transformed train set shape,"(390, 8)"
6,Transformed test set shape,"(98, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5128,0.3586,0.0000,0.0000,0.0000,-0.3209,-0.3216
1,0.6923,0.5069,0.1000,0.2500,0.1429,-0.0043,-0.0050
2,0.7179,0.5345,0.2000,0.4000,0.2667,0.1155,0.1261
3,0.6154,0.3552,0.2000,0.2222,0.2105,-0.0428,-0.0429
4,0.5128,0.2966,0.0000,0.0000,0.0000,-0.3209,-0.3216
5,0.6410,0.4513,0.1818,0.2857,0.2222,0.0036,0.0038
6,0.6667,0.4968,0.0909,0.2500,0.1333,-0.0201,-0.0241
7,0.5641,0.3214,0.1818,0.2000,0.1905,-0.1068,-0.1071
8,0.5897,0.3961,0.0000,0.0000,0.0000,-0.2140,-0.2404


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.9344,0.9353,0.8550,0.8960,0.8750,0.8306,0.8310


# Business Question:
What would happen if you could increase revenue by 10%?

In [64]:
x = reg_predictions_df['prediction_label'].sum()

print('Expected Sales: ${:,.2f}'.format(x))
print('10% increase in sales: ${:,.2f}'.format(x*.1))

Expected Sales: $5,833,864.00
10% increase in sales: $583,386.40
